In [1]:
import pickle, os
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm

In [2]:
# Data Download
with open('FS_손익계산서보정본', 'rb') as f:
    data = pickle.load(f)

with open('Final_FS_ind', 'rb') as f:
    ind = pickle.load(f)

In [26]:
# 특정 자산만 뽑아서 csv로 정리하기
def get_item(concept_id):
    item = []
    item_ind = []
    for i in range(len(data)):
        if type(data[i][0]) == type(data[0][0]):
            item.append(data[i][0][data[i][0][data[i][0].columns[0]] == concept_id])
            if len(data[i][0][data[i][0][data[i][0].columns[0]] == concept_id]) > 0:
                item_ind.append(ind[i])
    df_item = pd.concat(item)
    date = list(filter(lambda x: x[1] == ('연결재무제표',), df_item.columns))
    
    date_clean = list(map(lambda x: x[0], df_item[date].columns))
    df_item[date].columns = date_clean
    df_item2 = df_item[date]
    df_item2.columns = date_clean
    date_shifted = list(map(lambda x: shift_q(x), date_clean))
    df_item3 = df_item2
    df_item3.columns = date_shifted
    df_item4 = df_item3.groupby(level=0, axis=1).last()
    df_item3 = df_item4
    date_shifted2 = list(map(lambda x: shift_d(x), df_item4.columns))
    df_item3.columns = date_shifted2
    df_item5 = df_item3.groupby(level=0, axis=1).last()
    corp_code = list(map(lambda x:x[2], item_ind))
    df_item5['corp_code'] = corp_code
    df_item5 = df_item5.set_index('corp_code')
    return df_item5

def shift_q(date):
        if date[4:6]=='01' or date[4:6]=='02':
            date = date[:4] + '0331'
        elif date[4:6]=='04' or date[4:6]=='05':
            date = date[:4] + '0630'
        elif date[4:6]=='07' or date[4:6]=='08':
            date = date[:4] + '0930'
        elif date[4:6]=='10' or date[4:6]=='11':
            date = date[:4] + '1231'
        return date

def shift_d(date):
    # 케이스 두개 밖에 안되니까 그냥 수동으로 따져주자.
    if date[6:]=='01':
        date = date[:6] + '31'
    elif date[6:]=='29':
        date = date[:6] + '30'
    return date

In [30]:
equity = get_item('ifrs_Equity')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [101]:
get_item('ifrs_CurrentAssets')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,20091231,20100331,20100630,20100930,20101231,20110331,20110630,20110930,20111231,20120331,...,20190331,20190630,20190930,20191231,20200331,20200630,20200930,20201231,20210331,20210630
corp_code,,,,,,,,,,,,,,,,,,,,,
019660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
024840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.605511e+11,1.470452e+11,1.475676e+11,1.605771e+11,1.435739e+11,1.418619e+11,1.763016e+11,1.642513e+11,1.756979e+11,NaN
067830,7.167233e+10,NaN,NaN,NaN,9.255982e+10,NaN,NaN,NaN,7.687604e+10,NaN,...,1.076094e+11,1.109832e+11,8.828548e+10,9.546387e+10,7.647331e+10,8.692679e+10,6.674112e+10,NaN,NaN,NaN
101170,3.396121e+10,NaN,NaN,NaN,4.417977e+10,NaN,NaN,NaN,3.749360e+10,NaN,...,NaN,NaN,NaN,4.102700e+10,4.478048e+10,5.031517e+10,5.118101e+10,4.120120e+10,4.537005e+10,NaN
149950,NaN,NaN,NaN,NaN,1.119655e+10,NaN,NaN,NaN,1.997618e+10,NaN,...,NaN,NaN,NaN,3.921179e+10,NaN,3.636849e+10,4.006248e+10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010140,1.171384e+13,NaN,NaN,NaN,1.179578e+13,NaN,NaN,NaN,9.424202e+12,8.799282e+12,...,7.614103e+12,7.608548e+12,8.289874e+12,6.929474e+12,8.011572e+12,7.504618e+12,7.276993e+12,6.697963e+12,5.798443e+12,6.047272e+12
096530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.141449e+11,1.338536e+11,1.330247e+11,1.385021e+11,1.906689e+11,3.965241e+11,5.658282e+11,7.828743e+11,8.934696e+11,8.681539e+11
225190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.026482e+10,2.607931e+10,2.708931e+10,3.036160e+10,3.159486e+10,2.310333e+10,2.346744e+10,2.353333e+10,2.571993e+10,2.413528e+10


# Market return computation

In [45]:
def rm(date):
    price = pd.read_csv(date + '.csv', encoding = 'cp949')
    return (price['시가총액'] * price['등락률'] / price['시가총액'].sum()).sum()

def next_day(year, month, day):
    day +=1
    if day > 31:
        day=1
        month+=1
    if month > 12:
        month = 1
        year +=1
    return year, month, day

In [51]:
# 2013년 7월부터 2020년 6월까지 총 7년의 데이터를 구해보자.
year = 13
month = 7
day = 1
R_market = []
R_date = []
while year < 21:
    if str(year)+ '_' + str(month)+'_'+str(day)+'.csv' in csv_list:
        date = str(year)+ '_' + str(month)+'_'+str(day)
        R_market.append(rm(date))
        R_date.append(str(year)+ '_' + str(month)+'_'+str(day))
        year, month, day = next_day(year,month,day)
    else:
        year, month, day = next_day(year,month,day)

R_market = pd.DataFrame(R_market, index=R_date, columns = ['market_return'])

In [52]:
R_market = pd.DataFrame(R_market, index=R_date, columns = ['market_return'])

In [53]:
R_market

,market_return
13_7_1,-0.190980
13_7_2,-0.040313
13_7_3,-1.548017
13_7_4,0.794658
13_7_5,-0.258531
...,...
20_12_23,0.900136
20_12_24,1.662860
20_12_28,0.090903
20_12_29,0.959125


# Risk-free Return

In [56]:
os.chdir('C:\\Users\\lyoo9\\_Akros')
R_f = pd.read_excel('risk_free_rate.xls')

In [61]:
cd_91 = R_f.iloc[6,1:]

In [117]:
cd_91

시장금리 추이         2.68
Unnamed: 2      2.66
Unnamed: 3      2.66
Unnamed: 4      2.66
Unnamed: 5      2.65
                ... 
Unnamed: 97     0.69
Unnamed: 98     0.77
Unnamed: 99     0.98
Unnamed: 100    1.08
Unnamed: 101    1.17
Name: 6, Length: 101, dtype: object

# SMB

In [76]:
# 코스피 median 기준으로 나누자.
# 우선 매년 포트폴리오의 masking부터 진행하자.
os.chdir('C:\\Users\\lyoo9\\_System Trading\\Backend\\KRX_price')
year = 13
month = 6
day = 20
big_tables = []
small_tables = []
for i in range(7):
    not_last = True
    day = 20
    while not_last:
        day +=1
        if str(year)+ '_' + str(month)+'_'+str(day)+'.csv' in csv_list:
            this_day = day
        elif day > 31:
            not_last = False
    date = str(year)+ '_' + str(month)+'_'+str(this_day)
    smb_table = pd.read_csv(date + '.csv', encoding = 'cp949')
    median = smb_table[smb_table['시장구분'] == 'KOSPI']['시가총액'].median()
    big_tables.append(smb_table[smb_table['시가총액'] > median])
    small_tables.append(smb_table[smb_table['시가총액'] < median])
    year +=1

In [81]:
big_codes = [x['종목코드'] for x in big_tables]
small_codes = [x['종목코드'] for x in small_tables]

In [93]:
standard['종목코드']

0       001460
1       001465
2       035760
3       039720
4       017300
         ...  
1461    003280
1462    007630
1463    037440
1464    000500
1465    019640
Name: 종목코드, Length: 1466, dtype: object

In [94]:
big_codes[0]

0       060310
2       068400
3       006840
4       054620
6       138930
         ...  
1904    005010
1905    084110
1906    069260
1909    000540
1912    003280
Name: 종목코드, Length: 737, dtype: object

In [97]:
%time standard = pd.read_csv('00_6_30.csv', encoding = 'cp949')

Wall time: 29.9 ms


# Size Factor Replication

In [32]:
os.chdir('C:\\Users\\lyoo9\\_System Trading\\Backend\\KRX_price')

In [33]:
csv_list = os.listdir()

In [42]:
csv_list

['00_10_10.csv',
 '00_10_11.csv',
 '00_10_12.csv',
 '00_10_13.csv',
 '00_10_16.csv',
 '00_10_17.csv',
 '00_10_18.csv',
 '00_10_19.csv',
 '00_10_2.csv',
 '00_10_20.csv',
 '00_10_23.csv',
 '00_10_24.csv',
 '00_10_25.csv',
 '00_10_26.csv',
 '00_10_27.csv',
 '00_10_3.csv',
 '00_10_30.csv',
 '00_10_31.csv',
 '00_10_4.csv',
 '00_10_5.csv',
 '00_10_6.csv',
 '00_10_9.csv',
 '00_11_1.csv',
 '00_11_10.csv',
 '00_11_13.csv',
 '00_11_14.csv',
 '00_11_15.csv',
 '00_11_16.csv',
 '00_11_17.csv',
 '00_11_2.csv',
 '00_11_20.csv',
 '00_11_21.csv',
 '00_11_22.csv',
 '00_11_23.csv',
 '00_11_24.csv',
 '00_11_27.csv',
 '00_11_28.csv',
 '00_11_29.csv',
 '00_11_3.csv',
 '00_11_30.csv',
 '00_11_6.csv',
 '00_11_7.csv',
 '00_11_8.csv',
 '00_11_9.csv',
 '00_12_1.csv',
 '00_12_11.csv',
 '00_12_12.csv',
 '00_12_13.csv',
 '00_12_14.csv',
 '00_12_15.csv',
 '00_12_18.csv',
 '00_12_19.csv',
 '00_12_20.csv',
 '00_12_21.csv',
 '00_12_22.csv',
 '00_12_25.csv',
 '00_12_26.csv',
 '00_12_4.csv',
 '00_12_5.csv',
 '00_12_6.csv

In [34]:
standard = pd.read_csv('00_6_30.csv', encoding = 'cp949')

In [174]:
med = standard['시가총액'].median()

In [232]:
day = pd.read_csv('00_7_4.csv', encoding = 'cp949')
r = day['등락률']
r_win = winsorize(r, limits=[0.01, 0.01])

small_r = r_win[standard['시가총액'] < med]
big_r = r_win[standard['시가총액'] > med]
small_r.mean() - big_r.mean()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1468 but corresponding boolean dimension is 1466

In [186]:
big_r = r[standard['시가총액'] > med].mean()

small_r = r[standard['시가총액'] < med].mean()

In [227]:
small_r = r_win[standard['시가총액'] < med]
big_r = r_win[standard['시가총액'] > med]

In [219]:
r[standard['시가총액'] < med].max()

15.0

In [221]:
r[standard['시가총액'] > med].min()

-15.0

In [228]:
small_r.mean() - big_r.mean()

0.11614158829676446

In [199]:
r = day['등락률']

In [203]:
from scipy.stats.mstats import winsorize
r_win = winsorize(r, limits=[0.01, 0.01])

In [210]:
r_win = pd.Series(r_win)

In [176]:
standard[standard['시가총액'] < med]

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,001460,BYC,KOSPI,NaN,43100,-100,-0.23,42500,44250,42500,620,26777000,26920906500,624615
1,001465,BYC(1우),KOSPI,NaN,26300,-1600,-5.73,25100,28400,25100,1330,33849000,5664625500,215385
9,042300,KTB드림1,KOSDAQ,NaN,8640,-330,-3.68,0,0,0,0,0,11582784000,1340600
11,039370,KTB벤처2,KOSDAQ,NaN,4350,0,0.00,0,0,0,0,0,17857620000,4105200
12,038380,KTB자산배분2,KOSDAQ,NaN,3840,-10,-0.26,3840,3840,3840,140,537600,21623808000,5631200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,005725,흥아타이어(1우),KOSPI,NaN,3900,-30,-0.76,3880,3900,3860,2510,9732250,1404234000,360060
1461,003280,흥아해운,KOSPI,NaN,6100,100,1.67,6100,6100,6070,3090,18830100,5124000000,840000
1463,037440,희림,KOSDAQ,NaN,27850,-750,-2.62,28600,28600,27800,5705,160093250,27850000000,1000000
1464,000500,희성전선,KOSPI,NaN,4740,320,7.24,4520,4850,4520,176420,826797500,18486000000,3900000


In [226]:
med = standard[standard['시장구분'] == 'KOSPI']['시가총액'].median()

In [ ]:
standard